In [1]:
import pandas as pd
import numpy as np
import random

C:\Users\ASUS\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def load_Data(path):
    """读取并返回用户评分矩阵"""
    Data = pd.read_csv(path, nrows=10000)  # 由于数据过多，这里之前读取前25000行
    Ratings = Data.pivot_table(index='userId', columns='movieId', values='rating')
    return Ratings
UserItemMatrix = load_Data(r'G:\github项目\Recomendation_system\Data\movie\ratings.csv')

In [3]:
UserItemMatrix

movieId,1,2,3,4,5,6,7,8,10,11,...,182715,183611,184471,185031,185135,187541,187593,187595,188301,190183
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,NaN,4.0,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,...,4.0,4.0,3.5,4.5,5.0,NaN,4.0,4.0,NaN,NaN
63,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,4.0,NaN,3.5,NaN,NaN,4.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
class LFM(object):
    """构建隐语义模型类"""
    def __init__(self, Rating_Data, F, lr=0.0001, lmbd=0.000001, max_iteration=25):
        """
        Rating_Data: 用户评分矩阵
        F: 隐特征数量
        lr: 学习率
        lmbd: 正则化系数
        max_iteration: 最大迭代次数
        """
        self.F = F
        self.Rating_Data = Rating_Data
        self.lr = lr
        self.lmbd = lmbd
        self.max_iteration = max_iteration
        
        self.features = []
        for i in range(self.F):
            self.features.append("feature{}".format(i+1))
        user_id = Rating_Data.index
        item_id = Rating_Data.columns
        
        # 随机初始化用户和物品隐语义矩阵
        self.P = pd.DataFrame(np.random.rand(len(user_id), len(self.features)), index=user_id, columns=self.features)
        self.Q = pd.DataFrame(np.random.rand(len(self.features), len(item_id)), index=self.features, columns=item_id)
        
    def train(self):
        """使用随机梯度下降法训练用户和物品的隐矩阵"""
        for step in range(self.max_iteration):
            total_error = 0
            for uid in self.Rating_Data.index:
                random_iids = random.sample(list(self.Rating_Data.loc[uid].dropna().index), 1) # 因为不是所有的用户都评价了多个物品，这里只取一个物品进行随机梯度下降
                for random_iid in random_iids:
                    predict_point = self.predict(uid, random_iid)
                    true_point = self.Rating_Data.loc[uid].loc[random_iid]
                    user_error = true_point - predict_point
                    total_error += abs(user_error)
                    for f in range(self.F):
                        self.P.loc[uid].iloc[f] += self.lr * (user_error * self.Q.iloc[f].loc[random_iid] - 
                                                             self.lmbd * self.P.loc[uid].iloc[f])
                        self.Q.iloc[f].loc[random_iid] += self.lr * (user_error * self.P.loc[uid].iloc[f] - 
                                                            self.lmbd * self.Q.iloc[f].loc[random_iid])
            for iid in self.Rating_Data.columns:
                random_uids = random.sample(list(self.Rating_Data.loc[:, iid].dropna().index), 1)
                for random_uid in random_uids:
                    predict_point = self.predict(random_uid, iid)
                    true_point = self.Rating_Data.loc[random_uid].loc[iid]
                    item_error = true_point - predict_point
                    total_error += abs(item_error)
                    for f in range(self.F):
                        self.P.loc[random_uid].iloc[f] += self.lr * (item_error * self.Q.iloc[f].loc[iid] - 
                                                            self.lmbd * self.P.loc[random_uid].iloc[f])
                        self.Q.iloc[f].loc[iid] += self.lr * (item_error * self.P.loc[random_uid].iloc[f] - 
                                                            self.lmbd * self.Q.iloc[f].loc[iid])
            print("{}次迭代------------error:{}".format(step+1, total_error))
                    
    def predict(self, uid, iid):
        """返回指定用户对指定物品的预测评分"""
        return sum([self.P.loc[uid].iloc[f] * self.Q.iloc[f].loc[iid] for f in range(self.F)])

In [5]:
lfm = LFM(UserItemMatrix, 10)
lfm.train()
pre = lfm.predict(1, 1) # 实际值为4.0
pre # 这里效果不是很好，可以增加数据量或者是调整学习率和迭代次数

1次迭代------------error:7006.531481670675
2次迭代------------error:6914.186285687799
3次迭代------------error:6777.834301629341
4次迭代------------error:6800.927436047195
5次迭代------------error:6767.182697010026
6次迭代------------error:6491.182734625737
7次迭代------------error:6487.080508630959
8次迭代------------error:6480.182855728205
9次迭代------------error:6459.522531733305
10次迭代------------error:6416.515504016216
11次迭代------------error:6353.029992432144
12次迭代------------error:6296.107722436359
13次迭代------------error:6238.7929830573175
14次迭代------------error:6209.070769682552
15次迭代------------error:6139.81429912519
16次迭代------------error:6129.631230922175
17次迭代------------error:6110.247925725565
18次迭代------------error:6108.830443773523
19次迭代------------error:6041.237092248459
20次迭代------------error:5997.585870275137
21次迭代------------error:5984.458638313501
22次迭代------------error:6027.456259202743
23次迭代------------error:5932.2557709333905
24次迭代------------error:5919.421040999422
25次迭代------------error:5

2.508916135799344

In [8]:
class SVD(object):
    """带偏置的LFM"""
    def __init__(self, Rating_Data, F, lr=0.0005, lmbd=0.00001, max_iteration=50):
        self.F = F
        self.Rating_Data = Rating_Data
        self.lr = lr
        self.lmbd = lmbd
        self.max_iteration = max_iteration
        
        self.features = []
        for i in range(self.F):
            self.features.append('feature{}'.format(i+1))
        users_id = self.Rating_Data.index
        items_id = self.Rating_Data.columns
        
        # 初始化随机用户和物品隐语义矩阵
        self.P = pd.DataFrame(np.random.rand(len(users_id), len(self.features)), index=users_id, columns=self.features)
        self.Q = pd.DataFrame(np.random.rand(len(self.features), len(items_id)), index=self.features, columns=items_id)
        
        # 初始化偏置
        self.bu = pd.DataFrame(np.random.rand(len(users_id), 1), index=users_id, columns=['userbias'])
        self.bi = pd.DataFrame(np.random.rand(1, len(items_id)), index=['itembias'], columns=items_id)
        
        # 算出全局平均值
        Sum = 0
        count = 0
        for uid in users_id:
            for iid in items_id:
                if self.Rating_Data.loc[uid].loc[iid] > 0:
                    Sum += self.Rating_Data.loc[uid].loc[iid]
                    count += 1
        self.u = Sum / count
        
    def train(self):
        """随机梯度下降法更新用户和物品隐语义矩阵、用户和物品的偏置向量"""
        for step in range(self.max_iteration):
            total_error = 0
            for uid in self.Rating_Data.index:
                random_iids = random.sample(list(self.Rating_Data.loc[uid].dropna().index), 1)
                for random_iid in random_iids:
                    predict_point = self.predict(uid, random_iid)
                    true_point = self.Rating_Data.loc[uid].loc[random_iid]
                    user_error = true_point - predict_point
                    total_error += abs(user_error)
                    for f in range(self.F):
                        self.P.loc[uid].iloc[f] += self.lr * (user_error * self.Q.iloc[f].loc[random_iid] 
                                                              - self.lmbd * self.P.loc[uid].iloc[f])
                        self.Q.iloc[f].loc[random_iid] += self.lr * (user_error * self.P.loc[uid].iloc[f] 
                                                              - self.lmbd * self.Q.iloc[f].loc[random_iid])
                        self.bu.loc[uid] += self.lr * user_error - self.lmbd * self.bu.loc[uid]
            for iid in self.Rating_Data.columns:
                random_uids = random.sample(list(self.Rating_Data.loc[:, iid].dropna().index), 1)
                for random_uid in random_uids:
                    predict_point = self.predict(random_uid, iid)
                    true_point = self.Rating_Data.loc[random_uid].loc[iid]
                    item_error = true_point - predict_point
                    total_error += abs(item_error)
                    for f in range(self.F):
                        self.P.loc[random_uid].iloc[f] += self.lr * (item_error * self.Q.iloc[f].loc[iid] -
                                                             self.lmbd * self.P.loc[random_uid].iloc[f])
                        self.Q.iloc[f].loc[iid] += self.lr * (item_error * self.P.loc[random_uid].iloc[f] - 
                                                             self.lmbd * self.Q.iloc[f].loc[iid])
                        self.bi.loc[:,iid] += self.lr * item_error - self.lmbd * self.bi.loc[:,iid]
            print("{}次迭代-------error:{}".format(step+1, total_error))
            
    def predict(self, uid, iid):
        return self.u + self.bu.loc[uid].iloc[0] + self.bi.iloc[0].loc[iid] + sum([self.P.loc[uid].iloc[f] * self.Q.iloc[f].loc[iid] for f in range(self.F)])

In [9]:
svd = SVD(UserItemMatrix, 10)
svd.train()
pre1 = svd.predict(1, 1) # 实际值为4.0
pre1

1次迭代-------error:10956.88163927254
2次迭代-------error:9188.52727260016
3次迭代-------error:7921.540774215581
4次迭代-------error:6771.373173784517
5次迭代-------error:6114.772966598334
6次迭代-------error:5399.532501052777
7次迭代-------error:4944.506487380567
8次迭代-------error:4562.759500530098
9次迭代-------error:4391.75546787666
10次迭代-------error:4120.184739484768
11次迭代-------error:3950.3590790137587
12次迭代-------error:3788.48245787467
13次迭代-------error:3732.0858554351807
14次迭代-------error:3594.487911572407
15次迭代-------error:3561.7474263847525
16次迭代-------error:3475.512078789918
17次迭代-------error:3313.7112585469226
18次迭代-------error:3281.6426973385906
19次迭代-------error:3218.4297364126905
20次迭代-------error:3218.3596410279647
21次迭代-------error:3050.8030611362115
22次迭代-------error:3092.6895386094557
23次迭代-------error:3062.081654836335
24次迭代-------error:2963.3078468284625
25次迭代-------error:2982.348288605348
26次迭代-------error:2952.020187647578
27次迭代-------error:2911.3750033376887
28次迭代-------error:2899.547320

4.722918687229631

In [17]:
svd.predict(2, 1)

5.310093478190824